## Deep Learning Project: Sartorius cell segmentation Kaggle Challenge

We organised our code into multiple notebooks corresponding to the different experiments we performed.

Note: we didn't include the output folders (containing the .pth model weights files) because each folder takes multiple GB of disk space, but they are available upon request.

### Notebook 1: Training Mask R-CNN on the LIVECell dataset (pretraining)

In [1]:
import torch
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.structures import polygons_to_bitmask
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
dataDir=Path('../LIVECell_dataset_2021/images/livecell_train_val_images')
cfg = get_cfg()
# we'll train on the training and test datasets and validate the model on the validation dataset
register_coco_instances('sartorius_train',{}, '../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_val.json', dataDir)
register_coco_instances('sartorius_test',{}, '../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json', dataDir)

[12/25 01:16:39 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.07 seconds.
[12/25 01:16:39 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json


In [3]:
def polygon_to_rle(polygon, shape=(520, 704)):
    mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1])
    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

def precision_at(threshold, iou):
    # Calculate true positives, false positives and false negatives
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    # calculate the mAP score
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    enc_targs = [polygon_to_rle(enc_targ[0]) for enc_targ in enc_targs]
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    # Evaluator class for the Detectron2 model
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

#### Training the model
Note: the notebook crashed (out of memory) multiple times before the end of the training, so we continued the training on a Python console to save memory which allowed us to finish the 20000 iterations.

In [ ]:
# Define the model configuration and train
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train", "sartorius_test")
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 20000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print(cfg.OUTPUT_DIR)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
os.rename("output", "output_1.1")

[12/25 01:16:51 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.11 seconds.
[12/25 01:16:51 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[12/25 01:16:58 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 3.91 seconds.
[12/25 01:16:58 d2.data.datasets.coco]: Loaded 1564 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json
[12/25 01:17:09 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.41 seconds.
[12/25 01:17:10 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[12/25 01:17:17 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.03 second

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (9, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (9,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (32, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (32,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (8, 256, 1, 1) 

[12/25 01:17:43 d2.engine.train_loop]: Starting training from iteration 0


/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/25 01:17:48 d2.utils.events]:  eta: 0:55:07  iter: 19  total_loss: 6.403  loss_cls: 1.993  loss_box_reg: 0.4926  loss_mask: 0.6905  loss_rpn_cls: 2.933  loss_rpn_loc: 0.2866  time: 0.1753  data_time: 0.0237  lr: 9.9905e-06  max_mem: 6979M
[12/25 01:17:51 d2.utils.events]:  eta: 0:56:24  iter: 39  total_loss: 3.88  loss_cls: 1.898  loss_box_reg: 0.4774  loss_mask: 0.6825  loss_rpn_cls: 0.5755  loss_rpn_loc: 0.2799  time: 0.1737  data_time: 0.0023  lr: 1.998e-05  max_mem: 6979M
[12/25 01:17:55 d2.utils.events]:  eta: 0:57:20  iter: 59  total_loss: 3.578  loss_cls: 1.643  loss_box_reg: 0.5243  loss_mask: 0.6753  loss_rpn_cls: 0.3794  loss_rpn_loc: 0.3029  time: 0.1777  data_time: 0.0033  lr: 2.997e-05  max_mem: 6979M
[12/25 01:17:58 d2.utils.events]:  eta: 0:57:13  iter: 79  total_loss: 3.036  loss_cls: 1.161  loss_box_reg: 0.631  loss_mask: 0.6627  loss_rpn_cls: 0.3293  loss_rpn_loc: 0.2825  time: 0.1782  data_time: 0.0026  lr: 3.9961e-05  max_mem: 6979M
[12/25 01:18:02 d2.utils.even